In [1]:
!mkdir -p models
!wget https://github.com/Rikorose/DeepFilterNet/raw/main/models/DeepFilterNet2.zip
!unzip -o DeepFilterNet2.zip -d models
!rm -f DeepFilterNet2.zip

--2022-09-14 11:48:43--  https://github.com/Rikorose/DeepFilterNet/raw/main/models/DeepFilterNet2.zip
Resolving github.com (github.com)... 20.201.28.151
Connecting to github.com (github.com)|20.201.28.151|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Rikorose/DeepFilterNet/main/models/DeepFilterNet2.zip [following]
--2022-09-14 11:48:44--  https://raw.githubusercontent.com/Rikorose/DeepFilterNet/main/models/DeepFilterNet2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8629708 (8.2M) [application/zip]
Saving to: ‘DeepFilterNet2.zip’

DeepFilterNet2.zip  100%[===================>]   8.23M  9.59MB/s    in 0.9s    

2022-09-14 11:48:45 (9.59 MB/s) - ‘DeepFilterNet2.zip’ saved [8629708/862970

In [2]:
import torch
from df import config
from df.model import ModelParams
from df.enhance import enhance, init_df, load_audio, save_audio
import IPython.display as ipd
from df.utils import resample

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, df_state, _ = init_df("./models/DeepFilterNet2", config_allow_defaults=True)
model = model.to(device=device).eval()

2022-09-14 11:48:58 | INFO     | DF | Running on torch 1.12.1
2022-09-14 11:48:58 | INFO     | DF | Running on host MacBook-Pro-de-Mario.local
2022-09-14 11:48:58 | INFO     | DF | Git commit: 0a93933, branch: main
2022-09-14 11:48:58 | INFO     | DF | Loading model settings of DeepFilterNet2
2022-09-14 11:48:58 | INFO     | DF | Initializing model `deepfilternet2`
2022-09-14 11:48:58 | INFO     | DF | Found checkpoint ./models/DeepFilterNet2/checkpoints/model_96.ckpt.best with epoch 96
2022-09-14 11:48:58 | WARNING  | DF | Unexpected key: erb_comp.c
2022-09-14 11:48:58 | WARNING  | DF | Unexpected key: erb_comp.mn
2022-09-14 11:48:58 | INFO     | DF | Model loaded


In [4]:
file_input = "audio.mp3"
file_output = "audio-cleaned.wav"
file_input_wav = "audio.wav"

In [5]:
!ffmpeg -y -i {file_input} -vn -acodec pcm_s16le -ac 1 -f wav {file_input_wav}

ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbo

In [6]:
#ipd.Audio(file_input) # input audio

In [7]:
df_sr = ModelParams().sr # sample rate of the model
audio, meta = load_audio(file_input_wav, df_sr) # load audio with metadata
audio = enhance(model, df_state, audio)
audio = resample(audio, df_sr, meta.sample_rate)
save_audio(file_output, audio, sr=meta.sample_rate)

2022-09-14 11:48:59 | WARNING  | DF | Audio sampling rate does not match model sampling rate (44100, 48000). Resampling...


In [ ]:
#ipd.Audio(file_output) # output audio